# [作者 TsukiSama9292 - Github 連結](https://github.com/tsukisama9292)
## [參考文獻 - 連結](https://ianwu.tw/press/topic/command_line_program/yt-dlp.html#%E4%BB%8B%E7%B4%B9)

# 依賴安裝

In [ ]:
!pip install -q yt-dlp
!pip install -q -U gradio

In [ ]:
import yt_dlp
import threading
import queue
import os
import zipfile
import shutil
from datetime import datetime
import gradio as gr
# 全局變量或全局狀態，用於保存播放清單信息
playlist_info = {
    'total_videos': 0,
    'downloaded_videos': 0,
}
count = True

global ydl_opts

# 定義進度回調函數
def progress_hook(q):
    def hook(d):
        # 更新播放清單的下載進度
        global playlist_info
        global count
        if d['status'] == 'downloading':
            speed = d.get('speed', 0)
            speed_mb = speed / 1024 / 1024 if speed is not None else 0  # 將速度轉換為千字節每秒 (KB/s)，如果為 None，則設為 0
            q.put({
                '狀態': '下載中',
                '已下載大小(MB)': d['downloaded_bytes'] / 1024 / 1024,
                '當前影片總容量(MB)': d.get('total_bytes', 0) / 1024 / 1024,
                '網路速度(MB/秒)': speed_mb,
                '預設剩餘時間(秒)': d.get('eta', 0),
                '完成進度': f"{playlist_info['downloaded_videos']} / {playlist_info['total_videos']}"
            })
        elif d['status'] == 'finished':
            if(count):
                count = False
            else:
                count = True
                playlist_info['downloaded_videos'] += 1
            q.put({
                '狀態': '單項下載完畢',
                '檔名': d['filename'],
                '完成進度': f"{playlist_info['downloaded_videos']} / {playlist_info['total_videos']}"
            })
    return hook

# 定義下載函數
def download_Run(ydl_opts, url, q):
    ydl_opts['progress_hooks'] = [progress_hook(q)]

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        # 獲取播放清單的信息
        global playlist_info
        info = ydl.extract_info(url, download=False)
        if 'entries' in info:
            playlist_info['total_videos'] = len(info['entries'])
        ydl.download([url])

def download_Thread(ydl_opts,url):
    # 創建一個 Queue 來傳遞進度消息
    q = queue.Queue()

    # 在獨立的執行緒中執行下載函數
    thread = threading.Thread(target=download_Run, args=(ydl_opts, url, q))
    thread.start()

    # 即時顯示下載進度
    while thread.is_alive() or not q.empty():
        try:
            progress = q.get(timeout=1)
            yield progress
        except queue.Empty:
            pass
        
def re_ydl_opts_resolution(YOUTUBE_URL,resolution,out_path):
    # 初始化yt-dlp
    ydl = yt_dlp.YoutubeDL({'format': 'best'})
    # 檢查畫質格式是否可用
    info = ydl.extract_info(YOUTUBE_URL, download=False)
    formats = info.get('formats', [])
    global ydl_opts
    for format in formats:
        if format.get('format_id') == resolution:
            ydl_opts = {
            'format': f'{resolution}+bestaudio/best',
            'outtmpl': out_path, 
            }
            break
        else:
            ydl_opts = {
            'format': 'bestvideo+bestaudio/best',
            'outtmpl': out_path,
            }

def download_video(YOUTUBE_URL, do_zip,resolution):
    if(do_zip):
        out_path='Downloads_TMP/%(title)s.%(ext)s'
        re_ydl_opts_resolution(YOUTUBE_URL,resolution,out_path)
        yield from download_Thread(ydl_opts,YOUTUBE_URL)
        if has_files('Downloads_TMP'):
            zip_files('Video')
            yield "影片下載完成"
        else:
            yield "輸入的URL錯誤，無法獲取影片"
    else:
        out_path='Downloads/Video/%(title)s.%(ext)s'
        re_ydl_opts_resolution(YOUTUBE_URL,resolution,out_path)
        yield from download_Thread(ydl_opts,YOUTUBE_URL)
        if has_files('Downloads/Video'):
            yield "影片下載完成"
        else:
            yield "輸入的URL錯誤，無法獲取影片"
        
    

def download_audio(YOUTUBE_URL, do_zip):
    if(do_zip):
        ydl_opts = {
        'format': 'bestaudio/best',  # 下載最佳畫質
        'outtmpl': 'Downloads_TMP/%(title)s.%(ext)s',  # 影片文件名格式
        'postprocessors': [{  # 下載後處理選項
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        }
        yield from download_Thread(ydl_opts,YOUTUBE_URL)
        if has_files('Downloads_TMP'):
            zip_files('Audio')
            yield "音檔下載完成"
        else:
            yield "輸入的URL錯誤，無法獲取影片"
    else:
        # 音檔下載
        ydl_opts = {
            'format': 'bestaudio/best',  # 下載最佳畫質和音質，並合併
            'outtmpl': 'Downloads/Audio/%(title)s.%(ext)s',  # 影片文件名格式，下載到 downloads 資料夾
            'postprocessors': [{  # 下載後處理選項
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
        }
        yield from download_Thread(ydl_opts,YOUTUBE_URL)
        if has_files('Downloads/Audio'):
            yield "音檔下載完成"
        else:
            yield "輸入的URL錯誤，無法獲取音檔"
            
    
def has_files(folder_path):
    for _, _, files in os.walk(folder_path):
        if files:
            return True
    return False

def zip_files(Type):
    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day
    hour = now.hour
    minute = now.minute
    second = now.second
    folder_to_zip = 'Downloads_TMP'
    if has_files(folder_to_zip):
        zip_folder = 'Downloads_ZIP'
        if not os.path.exists(zip_folder):
            os.makedirs(zip_folder)
        
        zip_filename = f'{zip_folder}/{Type}_{year}_{month}_{day}_{hour}_{minute}_{second}.zip'
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for root, dirs, files in os.walk(folder_to_zip):
                for file in files:
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_to_zip))
        # 檢查資料夾是否存在
        if os.path.exists(folder_to_zip):
            shutil.rmtree(folder_to_zip)
    else:
        print(f"資料夾 {folder_to_zip} 中不包含檔案")
        
resolution_option={
'最高解析度':'bestvideo',
'4320p60/4320p':'571',
'2160p60':'315',
'2160p':'313',
'1440p60':'308',
'1440p':'271',
'1080p60':'303',
'1080p-Premium':'616',
'1080p':'248',
'720p60':'302',
'720p':'247',
'480p':'244',
'360p':'243',
'240p':'242',
'144p':'278',
}

def gradio_interface(function_choice, YOUTUBE_URL, do_zip,resolution):
    global playlist_info
    global count
    # 全局變量或全局狀態，用於保存播放清單信息
    playlist_info = {
        'total_videos': 0,
        'downloaded_videos': 0,
    }
    count = True
    if('playlist' not in YOUTUBE_URL):
        playlist_info['total_videos'] = 1
    if function_choice == "下載影片":
        yield from download_video(YOUTUBE_URL, do_zip,resolution_option[resolution])
    elif function_choice == "下載音檔":
        yield from download_audio(YOUTUBE_URL, do_zip)

def update_dropdown(function_choice):
    if function_choice == "下載影片":
        return gr.update(visible=True)
    else:
        return gr.update(visible=False)

# 使用 Gradio Blocks 來建立介面
with gr.Blocks() as iface:
    function_choice = gr.Radio(["下載影片", "下載音檔"], label="選擇功能")
    YOUTUBE_URL = gr.Textbox(label="輸入 YouTube 清單或影片的連結")
    do_zip = gr.Checkbox(label="壓縮成 ZIP 檔")
    resolution = gr.Dropdown(list(resolution_option.keys()), label="選擇解析度", visible=False)
    
    function_choice.change(fn=update_dropdown, inputs=function_choice, outputs=resolution)
    
    iface_interface = gr.Interface(
        fn=gradio_interface,
        inputs=[
            function_choice,
            YOUTUBE_URL,
            do_zip,
            resolution
        ],
        outputs="text"
    )
    
    #iface_interface.render()

iface.queue().launch(share=True)
# 4K 60fps 影片連結(測試用) : https://youtu.be/3aRJtJRa434
# 播放清單 和 影片 不可以放分享連結(share結尾)，要放一般的網址，如 : https://www.youtube.com/playlist?list=PLm2oaJc6zPi1ZJyHaGid4P3sCke4b0qfL

# 檢查解析度程式碼

In [ ]:
# 檢查解析度程式碼， ctrl + / ，可以 註解或取消註解 滑鼠框選的程式碼
# 除非未來有支援更高的解析度，不然 8K(4320p) 應該是 2024 年 YouTube 最高解析度了

# import yt_dlp

# def get_available_formats(url):
#     ydl_opts = {}
#     with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#         info = ydl.extract_info(url, download=False)
#         formats = info.get('formats', [])
#         return formats

# def print_formats(formats):
#     for f in formats:
#         print(f"ID: {f['format_id']}, Resolution: {f.get('resolution')}, Note: {f.get('format_note')}")

# url = 'https://youtu.be/PnfJDgS9VZc'
# formats = get_available_formats(url)
# print_formats(formats)

# 測試用URL
# 8K/60fps 'https://youtu.be/Y05_R_QNi88'
# 4K/60fps 'https://youtu.be/3aRJtJRa434'
# 1080p/60fps 'https://youtu.be/aA-PIDnq27E?list=LL'
# 1080p 'https://youtu.be/2GrPcspduGs?list=LL'
# 720p 'https://youtu.be/h1gEcaQzXtc?list=LL'

# 找到的固定ID通道
# Best bestvideo
# 4320p60/4320p 571
# 2160p60 315
# 2160p 313
# 1440p60 308
# 1440p 271
# 1080p60 303
# 1080p-Premium 616
# 1080p 248
# 720p60 302
# 720p 247
# 480p 244
# 360p 243
# 240p 242
# 144p 278